In [6]:
from pathlib import Path
import json
import re
from tqdm import tqdm


settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['단어별 방언 식별', '방언 종류 예측', '대화 주제 식별', '화자 인식', '화자 관계 식별', 'LM(한국어)', 'dialog', '번역(방언-표준어)', '번역(표준어-방언)']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
def remove_nested_brackets(text):

    # 중괄호, 소괄호, 대괄호 모두를 제거하기 위한 패턴
    brackets_pattern = re.compile(r'\{[^{}]*\}|\([^()]*\)|\[[^\[\]]*\]')

    # &로 둘러싸인 단어 제거 패턴 추가
    special_words_pattern = re.compile(r'&[^&]*&')

    while True:
        new_text = brackets_pattern.sub('', text)
        if new_text == text:  # 변경 사항이 없으면 반복 중단
            break
        text = new_text

    text = special_words_pattern.sub('', text)

    return text

#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)

            #### data preproces

            filtered_utterances = []
            for utt in line['utterance']:
                fields = ['standard_form', 'dialect_form']
                for field in fields:
                    if field in utt:

                        cleaned_text = remove_nested_brackets(utt[field])

                        utt[field] = cleaned_text
                filtered_utterances.append(utt)

            line['utterance'] = filtered_utterances

            #### data preprocess end 

            #### 단어별 방언 식별
            data = {'input': None, 'output': None}
            ## preprocess data from line
            in1=[]
            ou1=[]
            for utterance in line['utterance']:
                result = {
                    'id': utterance['id'],
                    'standard_form': utterance['standard_form'],
                    'eojeolList': [{'id': eojeol['id'], 'eojeol': eojeol['eojeol']} for eojeol in utterance['eojeolList']]
                }
                in1.append(result)

            for utterance in line['utterance']:
                eojeol_list = {
                        'id': utterance['id'],
                        'eojeolList': [{'standard': eojeol['standard'], 'isDialect': eojeol['isDialect']} for eojeol in utterance['eojeolList']]
                    }
                ou1.append(eojeol_list)
            
            data['input']=in1
            data['output']=ou1
            ## preprocess data from line end
            #task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 단어별 방언 식별 end


            #### 방언 종류 예측
            data = {'input': None, 'output': None}
            in2=[]
            ou2=[]
            ## preprocess data from line
            for utterance in line['utterance']:
                result = {
                    'id': utterance['id'],
                    'dialect_form': utterance['dialect_form'],
                    'eojeolList': [{'id': eojeol['id'], 'eojeol': eojeol['eojeol']} for eojeol in utterance['eojeolList']]
                }
                in2.append(result)
                ou2.append(any([eo['isDialect'] for eo in utterance['eojeolList']]))

            data['input'] = in2

            dialect = line['metadata']['category'].split('>')[0].strip()
            data['output'] = ou2

            ## preprocess data from line end
            #task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 방언 종류 예측 end




            #### 대화 주제 식별
            data = {'input': None, 'output': None}
            ## preprocess data from line
            
            in3=[]
            topic = line['metadata']['topic']
            for utterance in line['utterance']:
                result = {
                    'standard_form': utterance['standard_form'],
                }
                in3.append(result)
                ## preprocess data from line end

            data['input']=in3
            data['output']=topic

            #task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
            print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 대화 주제 식별 end



            #### 화자 인식
            data = {'input': None, 'output': None}
            ## preprocess data from line

            in4 = []
            ou4 = []
            speakers = {speaker['id']: speaker for speaker in line['speaker']}
            
            for utterance in line['utterance']:
                speaker_id = utterance['speaker_id']
                speaker_info = speakers.get(speaker_id, {})

                result = {
                    'age': speaker_info.get('age', 'Unknown'),
                    'occupation': speaker_info.get('occupation', 'Unknown'),
                    'sex': speaker_info.get('sex', 'Unknown'),
                    'birthplace': speaker_info.get('birthplace', 'Unknown'),
                    'principal_residence': speaker_info.get('principal_residence', 'Unknown'),
                    'current_residence': speaker_info.get('current_residence', 'Unknown'),
                    'education': speaker_info.get('education', 'Unknown')
                }
                in4.append(result)
                
            for utterance in line['utterance']:
                speaker_id = utterance['speaker_id']
                speaker_info = speakers.get(speaker_id, {})

                result = speaker_id
                ou4.append(result)


            data['input']=in4
            data['output']=ou4
            ## preprocess data from line end
            #task_files[3].write(json.dumps(data, ensure_ascii=False)+'\n')
            print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 화자 인식 end



            #### 화자 관계 식별
            data = {'input': None, 'output': None}
            ## preprocess data from line
            in5=[]
            ou5=[]
            speakers = {speaker['id']: speaker for speaker in line['speaker']}
            for utterance in line['utterance']:
                result = {
                    'id': utterance['id'],
                    'standard_form': utterance['standard_form'],
                    'speaker_id': utterance['speaker_id'] 
                }
                

                speaker_id = utterance['speaker_id']
                speaker_info = speakers.get(speaker_id, {})

                result['speaker_id'] = speaker_id
                result['speaker_age'] = speaker_info.get('age', 'Unknown')
                result['speaker_occupation'] = speaker_info.get('occupation', 'Unknown')
                result['speaker_sex'] = speaker_info.get('sex', 'Unknown')

                in5.append(result)


            if 'setting' in line:
                setting_relation = line['setting'].get('relation')
                result = {
                    'relation': setting_relation
                }

                ou5.append(result)

            data['input']=in5
            data['output']=ou5
            ## preprocess data from line end
            #task_files[4].write(json.dumps(data, ensure_ascii=False)+'\n')
            print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 화자 관계 식별 end


            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line

            in6 = []
            for utterance in line['utterance']:
                result = utterance['dialect_form']
                in6.append(result)

            text1 = '\n'.join(in6)

            data['text'] = text1

            ## preprocess data from line end
            #task_files[5].write(json.dumps(data, ensure_ascii=False)+'\n')
            print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(한국어) end


            #### dialog
            data = {'text': None}
            ## preprocess data from line
            
            data['text'] = {}
            data['text']['metadata'] = line['metadata']
            data['text']['speaker'] = line['speaker']
            data['text']['dialog'] = []  # 새로운 dialog 리스트를 추가

            for utt in line['utterance']:
                # speaker_id와 form만 포함하는 새로운 딕셔너리 생성
                new_utt = {'speaker_id': utt['speaker_id'], 'dialect_form': utt['dialect_form']}
                data['text']['dialog'].append(new_utt)

            ## preprocess data from line end
            #task_files[6].write(json.dumps(data, ensure_ascii=False)+'\n')
            print(json.dumps(data, indent=4, ensure_ascii=False))
            #### dialog end




            #### 번역(방언-표준어)
            data = {'input': None, 'output': None}
            ## preprocess data from line

            in7 = []
            ou7 = []
            
            for utterance in line['utterance']:
                result = {
                    'dialect_form': utterance['dialect_form']
                }
                in7.append(result)
                

            for utterance in line['utterance']:
                result = {
                    'standard_form': utterance['standard_form']
                }
                ou7.append(result)
            data['input']=in7
            data['output']=ou7
            ## preprocess data from line end
            #task_files[7].write(json.dumps(data, ensure_ascii=False)+'\n')
            print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 번역(방언-표준어) end




            #### 번역(표준어-방언)
            data = {'input': None, 'output': None}
            ## preprocess data from line
            in8 = []
            ou8 = []
            for utterance in line['utterance']:
                result = {
                    'standard_form': utterance['standard_form']
                }
                in8.append(result)
                

            for utterance in line['utterance']:
                result = {
                    'dialect_form': utterance['dialect_form']
                }
                ou8.append(result)
            data['input']=in8
            data['output']=ou8

            ## preprocess data from line end
            #task_files[8].write(json.dumps(data, ensure_ascii=False)+'\n')
            print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 번역(표준어-방언) end


            break
        break
    break

    for path in task_files:
        path.close()      
        
#### task preprocess end    

train:   0%|          | 0/1 [00:00<?, ?it/s]

train:   0%|          | 0/1 [01:03<?, ?it/s]

{
    "input": [
        {
            "id": "DGDQ20000020.1.1.1",
            "standard_form": "글쎄요",
            "eojeolList": [
                {
                    "id": 1,
                    "eojeol": "글쎄요"
                }
            ]
        },
        {
            "id": "DGDQ20000020.1.1.2",
            "standard_form": "예?",
            "eojeolList": [
                {
                    "id": 1,
                    "eojeol": "예?"
                }
            ]
        },
        {
            "id": "DGDQ20000020.1.1.3",
            "standard_form": "뭔 얘기부터 할까요?",
            "eojeolList": [
                {
                    "id": 1,
                    "eojeol": "뭔"
                },
                {
                    "id": 2,
                    "eojeol": "얘기부터"
                },
                {
                    "id": 3,
                    "eojeol": "할까유?"
                }
            ]
        },
        {
            "id": "DGDQ20000020.1.1.4",
   

In [7]:
in6

['글쎄요',
 '예?',
 '뭔 얘기부터 할까유?',
 '바다 얘기 좀 해주세요 ',
 '바다에요?',
 '네.',
 '바다의 무슨 얘기를 해야 되나?',
 '아니 언제부터 해녀일 하셨어요?',
 '해녀 일은요 옛날부터 했는데요.',
 '네.',
 '옛날에 바다에서 내가 잘한다고 엠비씨까지 갔다 왔거든요 제가.',
 '제가 그때 맻 년도인가?',
 '편하게 해 편하게요.',
 '구십 몇 년 도에 했는데요?',
 '그때가 엠비씨 방송국에서 해녀 하라고 또 인터뷰 하자 하대?',
 '그래서 엠비씨 방송 공개 홀 강릉 가 가지고 노래도 부르구',
 '근데 내가 그때는 내가 지금 이렇게 말이 빨른지도 몰르구 사투린지도 몰랐어.',
 '근데 그 -티- 티비에 나오는데 보니까',
 '세상에 내가 그렇게 말이 빨르고 사투리를 하는 거야.',
 '얼마나 우리가  테베리 보면서 웃었는지 몰라.',
 '어.',
 '노래도 막 누가 나 기억도 난다.',
 '지금도 누가 울어도 하고 저 명파 - 그- 그 사람 죽은 사람 그 누구야?',
 '그 사람하고 같이 그때 엠비씨 가서 그것도 했어.',
 '근데 그 해녀라는 직업이 좀 그렇더라고?',
 '그때 나오면서도 물론 뭐',
 '-어떠- 어떠한 자신감도 있겠지만',
 '네 그래서?',
 '또 연결 되는 거야?',
 '어 말이 빨리 돼서 어떻게 했다고?',
 '말이 빨리 되니까 듣기가 싫 듣기가 안 좋더라고.',
 '그런 게  그런 게 ',
 '-형- 형님 말 빨라.',
 '말 빨르구 목소리 크고',
 '어 목소리 크구 말 빨르구 누구한테 하면 화낸 거 같애서',
 ' 나는 또 일본도 한번 견학을 갔는데',
 '일본 그 저 저 저기 내가 그때 뭐  해녀 회장 할 때다.',
 '일본을 갔는데 하도 목소리가 크니까 어항 협회에서 보내 줬거든?',
 '이제 전국 인제 해녀들 부인 회원들 뭐 어촌 계장들 이렇게 해서',
 '그 뭐 오사카성도 가고 그때 막 그랬어.',
 '근데 글쎄 거기 갔는데도 내가 목소리가 큰 거야?',
 '식당에 가니까 